In [145]:
import sys
import os
import importlib

parent = os.path.dirname(os.getcwd())
sys.path.insert(0, parent)

import numpy as np
import torch
import pandas as pd
import matplotlib.pyplot as plt

from Helpers import loadData, multiplot, sphereMask, dataSizes
importlib.reload(sys.modules['Helpers.sphereMask'])
importlib.reload(sys.modules['Helpers.dataSizes'])
importlib.reload(sys.modules['Helpers'])

<module 'Helpers' from 'c:\\Users\\Lab User\\Desktop\\temp1\\Granular-Compression\\Identification\\Helpers\\__init__.py'>

In [146]:
folderData = os.path.join(os.path.dirname(parent),'Data')
data = loadData(location=folderData, fileName='downscale_17.hdf5')

In [149]:
device = torch.cuda.get_device_name()
desktopGPU = 'NVIDIA RTX A2000 12GB'
serverGPU = 'NVIDIA RTX 4000 Ada Generation'

if device == desktopGPU:
    client = 'Desktop'
    cuts = 2
elif device == serverGPU:
    client = 'Server'
    cuts = 1

netVram = torch.cuda.get_device_properties(0).total_memory
resVram = torch.cuda.memory_reserved(0)
aloVram = torch.cuda.memory_allocated(0)
info = [netVram,resVram,aloVram]

df = pd.DataFrame([np.round(item/(1024**3),2) for item in info],
                  index=['Total','Reserved','Allocated'],columns=['VRam'])

print(f'{client} device detected: {device}')
print(f'Data Ram: {np.round(data.nbytes/(1024**3),2)} GB')
display(df)


Desktop device detected: NVIDIA RTX A2000 12GB
Data Ram: 0.21 GB


,VRam
Total,11.99
Reserved,0.00
Allocated,0.00


In [ ]:
r,c,z = data.shape
kernel = sphereMask(diameter=100,pad=2,scale=.6)
pad = int(kernel.shape[0]/2)

lower = data[:,:,0:int(z/2)+pad] # Lower Half
upper = data[:,:,int(z/2)-pad:z] # Upper Half

if cuts==1:
    chunks = [lower,upper]
elif cuts == 2:
    chunks = [lower[:,0:int(c/2)+pad,:],
              lower[:,int(c/2)-pad:c,:],
              upper[:,0:int(c/2)+pad,:],
              upper[:,int(c/2)-pad:c,:]]

In [167]:
import sys
import os
import importlib

parent = os.path.dirname(os.getcwd())
sys.path.insert(0, parent)

import numpy as np
import torch
import pandas as pd
import matplotlib.pyplot as plt

from Helpers import loadData, multiplot, sphereMask, dataSizes, chunkor
importlib.reload(sys.modules['Helpers.sphereMask'])
importlib.reload(sys.modules['Helpers.chunkor'])
importlib.reload(sys.modules['Helpers'])

<module 'Helpers' from 'c:\\Users\\Lab User\\Desktop\\temp1\\Granular-Compression\\Identification\\Helpers\\__init__.py'>

In [168]:
chunks = chunkor(data,kernel)

Desktop device detected: NVIDIA RTX A2000 12GB
Data Ram: 0.21 GB
Attempting 4 partitions


,VRam
Total,11.99
Reserved,0.00
Allocated,0.00
